In [1]:
import pandas as pd
import numpy as np
import csv

import torch

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.jp-Cell { margin-left: -45% !important; margin-right: -45% !important; }</style>"))

In [3]:
def assign_computing_load(utilization):
    if utilization <= 20:
        return 1
    elif utilization <= 40:
        return 2
    elif utilization <= 60:
        return 3
    elif utilization <= 80:
        return 4
    else:
        return 5

In [4]:
#data = pd.read_csv('tp_gpu_workload_only_log.csv')
data = pd.read_csv('violet_gpu_allocation_data.csv')
log = pd.read_csv('violet_gpu_allocation_data.csv')

data['start'] = pd.to_datetime(data['start'], utc=True)
data['finish'] = pd.to_datetime(data['finish'], utc=True)
data['time_diff'] = data['finish'] - data['start']
data

,node,gpu_number,GPU,gpu_utilization,pod_name,pod_type,project,namespace,user_team,start,finish,pod_type_color,project_color,time_diff
0,gpu-server01,0,gpu-server01:0,68.499391,violet-run-pipeline-gpu-kb27t-3379495359,task,PROJECT_17,ns-16904374776309414,NaN,2023-11-09 07:30:00+00:00,2023-11-12 16:45:00+00:00,red,"(0.29545293465748007, 0.0, 1.0, 1.0)",3 days 09:15:00
1,gpu-server01,0,gpu-server01:0,0.000000,violet-run-pipeline-gpu-ff44j-3079852243,task,PROJECT_1,ns-16904458533318198,NaN,2023-11-13 05:45:00+00:00,2023-11-13 06:30:00+00:00,red,"(1.0, 0.0, 0.0, 1.0)",0 days 00:45:00
2,gpu-server01,0,gpu-server01:0,34.499490,violet-run-pipeline-gpu-xnpsz-830121636,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-14 05:45:00+00:00,2023-12-20 23:45:00+00:00,red,"(0.0, 0.7784117755708665, 1.0, 1.0)",36 days 18:00:00
3,gpu-server01,0,gpu-server01:0,93.161125,violet-run-pipeline-gpu-fftp4-1617571953,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-12-22 05:00:00+00:00,2023-12-27 05:15:00+00:00,red,"(0.0, 0.7784117755708665, 1.0, 1.0)",5 days 00:15:00
4,gpu-server01,0,gpu-server01:0,71.150017,violet-run-pipeline-gpu-9tnd8-1865815998,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-12-27 05:30:00+00:00,2024-01-11 21:45:00+00:00,red,"(0.0, 0.7784117755708665, 1.0, 1.0)",15 days 16:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,gpu-server21,3,gpu-server21:3,0.352398,instance-2142-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-20 00:30:00+00:00,2023-12-21 00:30:00+00:00,blue,"(0.0, 0.5099455980137799, 1.0, 1.0)",1 days 00:00:00
2057,gpu-server21,3,gpu-server21:3,0.009993,instance-2242-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-21 08:15:00+00:00,2023-12-22 08:45:00+00:00,blue,"(0.0, 0.5099455980137799, 1.0, 1.0)",1 days 00:30:00
2058,gpu-server21,3,gpu-server21:3,0.000000,instance-2304-0,instance,PROJECT_14,ns-1693886715473508,TEAM_11,2023-12-26 05:45:00+00:00,2023-12-26 06:00:00+00:00,blue,"(0.0, 0.5099455980137799, 1.0, 1.0)",0 days 00:15:00
2059,gpu-server21,3,gpu-server21:3,0.000000,instance-2312-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-27 07:30:00+00:00,2023-12-27 08:15:00+00:00,blue,"(0.0, 0.5099455980137799, 1.0, 1.0)",0 days 00:45:00


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   node             2061 non-null   object             
 1   gpu_number       2061 non-null   int64              
 2   GPU              2061 non-null   object             
 3   gpu_utilization  2061 non-null   float64            
 4   pod_name         2061 non-null   object             
 5   pod_type         2050 non-null   object             
 6   project          2061 non-null   object             
 7   namespace        2061 non-null   object             
 8   user_team        1965 non-null   object             
 9   start            2061 non-null   datetime64[ns, UTC]
 10  finish           2061 non-null   datetime64[ns, UTC]
 11  pod_type_color   2061 non-null   object             
 12  project_color    2061 non-null   object             
 13  time_diff        2

In [6]:
#task_log = log
print("Total Count: " + str(len(log)))
task_log = log[log['pod_type'].isin({'task'})]
insta_log = log[log['pod_type'].isin({'instance'})]
print("Only task: " + str(len(task_log)))
print("\nUnique Count\n")
print('pod_name: ' + str(len(task_log['pod_name'].unique())))
print('namespace: ' + str(len(task_log['namespace'].unique())))
print('start: ' + str(len(task_log['start'].unique())))

Total Count: 2061
Only task: 808

Unique Count

pod_name: 368
namespace: 11
start: 335


In [7]:
def assign_flavor(node):
    if node.startswith('gpu-server0') and node[10:].isdigit() and 1 <= int(node[10:]) <= 7:
        return 'A100'
    else:
        return 'A30'

In [8]:
task_group = pd.DataFrame(task_log)
task_group['count'] = task_group.groupby('pod_name')['pod_name'].transform('count')
task_group = task_group.sort_values(by='start', ascending=True)


task_group['start'] = pd.to_datetime(data['start'], utc=True)
task_group['finish'] = pd.to_datetime(data['finish'], utc=True)
task_group['time_diff'] = task_group['finish'] - task_group['start']
task_group['computing_load'] = task_group['gpu_utilization'].apply(assign_computing_load)
task_group['flavor'] = task_group['node'].apply(assign_flavor)


insta_log = pd.DataFrame(insta_log)

insta_log['start'] = pd.to_datetime(data['start'], utc=True)
insta_log['finish'] = pd.to_datetime(data['finish'], utc=True)
insta_log['time_diff'] = insta_log['finish'] - insta_log['start']
insta_log['computing_load'] = insta_log['gpu_utilization'].apply(assign_computing_load)
insta_log['count'] = 1
insta_log['flavor'] = insta_log['node'].apply(assign_flavor)

task_group = task_group.drop(columns=['node','gpu_number','GPU','project_color', 'pod_type_color'])
insta_log = insta_log.drop(columns=['node','gpu_number','GPU','project_color', 'pod_type_color'])

task_group

,gpu_utilization,pod_name,pod_type,project,namespace,user_team,start,finish,count,time_diff,computing_load,flavor
151,97.469773,violet-run-pipeline-gpu-9fg4g-4198045470,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-12-16 00:45:00+00:00,4,37 days 12:30:00,5,A100
465,93.878238,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 02:15:00+00:00,8,13 days 14:00:00,5,A100
486,94.111422,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,A100
499,93.718447,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 08:00:00+00:00,8,13 days 19:45:00,5,A100
518,94.085403,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,A100
...,...,...,...,...,...,...,...,...,...,...,...,...
186,92.666223,violet-run-pipeline-gpu-9bdds-2661085165,task,PROJECT_13,ns-16904409679649498,TEAM_1,2024-02-05 04:30:00+00:00,2024-02-07 15:00:00+00:00,2,2 days 10:30:00,5,A100
143,0.233444,violet-run-pipeline-gpu-vd7wk-4198087334,task,PROJECT_13,ns-16904409679649498,TEAM_1,2024-02-06 07:00:00+00:00,2024-02-06 23:30:00+00:00,1,0 days 16:30:00,1,A100
134,10.524943,violet-run-pipeline-gpu-9w7hx-1761467166,task,PROJECT_13,ns-16904409679649498,TEAM_1,2024-02-06 07:00:00+00:00,2024-02-06 12:45:00+00:00,2,0 days 05:45:00,1,A100
326,17.333392,violet-run-pipeline-gpu-9w7hx-1761467166,task,PROJECT_13,ns-16904409679649498,TEAM_1,2024-02-07 02:15:00+00:00,2024-02-07 15:00:00+00:00,2,0 days 12:45:00,1,A100


In [9]:
task_total = task_group
task_total = pd.concat([task_group, insta_log], ignore_index=True)
task_total

,gpu_utilization,pod_name,pod_type,project,namespace,user_team,start,finish,count,time_diff,computing_load,flavor
0,97.469773,violet-run-pipeline-gpu-9fg4g-4198045470,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-12-16 00:45:00+00:00,4,37 days 12:30:00,5,A100
1,93.878238,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 02:15:00+00:00,8,13 days 14:00:00,5,A100
2,94.111422,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,A100
3,93.718447,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 08:00:00+00:00,8,13 days 19:45:00,5,A100
4,94.085403,violet-run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,A100
...,...,...,...,...,...,...,...,...,...,...,...,...
2045,0.352398,instance-2142-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-20 00:30:00+00:00,2023-12-21 00:30:00+00:00,1,1 days 00:00:00,1,A30
2046,0.009993,instance-2242-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-21 08:15:00+00:00,2023-12-22 08:45:00+00:00,1,1 days 00:30:00,1,A30
2047,0.000000,instance-2304-0,instance,PROJECT_14,ns-1693886715473508,TEAM_11,2023-12-26 05:45:00+00:00,2023-12-26 06:00:00+00:00,1,0 days 00:15:00,1,A30
2048,0.000000,instance-2312-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-27 07:30:00+00:00,2023-12-27 08:15:00+00:00,1,0 days 00:45:00,1,A30


In [10]:
task_total['pod_name'] = task_total['pod_name'].replace('violet-', '', regex=True)
task_total

,gpu_utilization,pod_name,pod_type,project,namespace,user_team,start,finish,count,time_diff,computing_load,flavor
0,97.469773,run-pipeline-gpu-9fg4g-4198045470,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-12-16 00:45:00+00:00,4,37 days 12:30:00,5,A100
1,93.878238,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 02:15:00+00:00,8,13 days 14:00:00,5,A100
2,94.111422,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,A100
3,93.718447,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 08:00:00+00:00,8,13 days 19:45:00,5,A100
4,94.085403,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,A100
...,...,...,...,...,...,...,...,...,...,...,...,...
2045,0.352398,instance-2142-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-20 00:30:00+00:00,2023-12-21 00:30:00+00:00,1,1 days 00:00:00,1,A30
2046,0.009993,instance-2242-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-21 08:15:00+00:00,2023-12-22 08:45:00+00:00,1,1 days 00:30:00,1,A30
2047,0.000000,instance-2304-0,instance,PROJECT_14,ns-1693886715473508,TEAM_11,2023-12-26 05:45:00+00:00,2023-12-26 06:00:00+00:00,1,0 days 00:15:00,1,A30
2048,0.000000,instance-2312-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-27 07:30:00+00:00,2023-12-27 08:15:00+00:00,1,0 days 00:45:00,1,A30


In [11]:
first_column = task_total.pop('gpu_utilization')  # 첫 번째 열을 선택하고 DataFrame에서 제거
task_total['gpu_utilization'] = first_column  # 선택한 열을 DataFrame의 마지막 열로 추가
first_column = task_total.pop('flavor')  # 첫 번째 열을 선택하고 DataFrame에서 제거
task_total['flavor'] = first_column  # 선택한 열을 DataFrame의 마지막 열로 추가

task_total

,pod_name,pod_type,project,namespace,user_team,start,finish,count,time_diff,computing_load,gpu_utilization,flavor
0,run-pipeline-gpu-9fg4g-4198045470,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-12-16 00:45:00+00:00,4,37 days 12:30:00,5,97.469773,A100
1,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 02:15:00+00:00,8,13 days 14:00:00,5,93.878238,A100
2,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,94.111422,A100
3,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 08:00:00+00:00,8,13 days 19:45:00,5,93.718447,A100
4,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 01:30:00+00:00,8,13 days 13:15:00,5,94.085403,A100
...,...,...,...,...,...,...,...,...,...,...,...,...
2045,instance-2142-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-20 00:30:00+00:00,2023-12-21 00:30:00+00:00,1,1 days 00:00:00,1,0.352398,A30
2046,instance-2242-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-21 08:15:00+00:00,2023-12-22 08:45:00+00:00,1,1 days 00:30:00,1,0.009993,A30
2047,instance-2304-0,instance,PROJECT_14,ns-1693886715473508,TEAM_11,2023-12-26 05:45:00+00:00,2023-12-26 06:00:00+00:00,1,0 days 00:15:00,1,0.000000,A30
2048,instance-2312-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-27 07:30:00+00:00,2023-12-27 08:15:00+00:00,1,0 days 00:45:00,1,0.000000,A30


In [12]:
task_total = task_total.drop_duplicates(subset='pod_name', keep='first')
task_total

,pod_name,pod_type,project,namespace,user_team,start,finish,count,time_diff,computing_load,gpu_utilization,flavor
0,run-pipeline-gpu-9fg4g-4198045470,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-12-16 00:45:00+00:00,4,37 days 12:30:00,5,97.469773,A100
1,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 02:15:00+00:00,8,13 days 14:00:00,5,93.878238,A100
6,run-pipeline-gpu-qr2fk-3879037461,task,PROJECT_5,ns-1690440796148727,NaN,2023-11-08 12:15:00+00:00,2023-11-13 00:15:00+00:00,1,4 days 12:00:00,4,67.799001,A100
7,run-pipeline-gpu-cqxgz-1528876722,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-11-27 07:30:00+00:00,16,18 days 19:15:00,3,53.086390,A100
17,run-pipeline-gpu-8z2pg-3801962406,task,PROJECT_1,ns-16904458533318198,TEAM_2,2023-11-08 12:15:00+00:00,2023-11-09 09:15:00+00:00,2,0 days 21:00:00,5,95.259751,A30
...,...,...,...,...,...,...,...,...,...,...,...,...
2037,instance-2141-0,instance,PROJECT_14,ns-1693886715473508,TEAM_6,2023-12-01 09:00:00+00:00,2023-12-02 09:00:00+00:00,1,1 days 00:00:00,1,0.000000,A30
2039,instance-2220-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-08 08:30:00+00:00,2023-12-09 08:30:00+00:00,1,1 days 00:00:00,1,0.000000,A30
2044,instance-2260-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-18 10:00:00+00:00,2023-12-18 10:15:00+00:00,1,0 days 00:15:00,1,0.000000,A30
2045,instance-2142-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-20 00:30:00+00:00,2023-12-21 00:30:00+00:00,1,1 days 00:00:00,1,0.352398,A30


In [13]:
y_ratio = 0.7
n_ratio = 0.3
data_length = len(task_total)

# y와 n의 개수 계산
num_y = int(data_length * y_ratio)
num_n = data_length - num_y

# y와 n의 리스트 생성
y_values = ['y'] * num_y
n_values = ['n'] * num_n

# y와 n의 리스트 결합
values = y_values + n_values

# 리스트를 무작위로 섞기
np.random.shuffle(values)
task_total.loc[:,'preemption'] = values
task_total

/tmp/ipykernel_14935/2252511449.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_total.loc[:,'preemption'] = values


,pod_name,pod_type,project,namespace,user_team,start,finish,count,time_diff,computing_load,gpu_utilization,flavor,preemption
0,run-pipeline-gpu-9fg4g-4198045470,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-12-16 00:45:00+00:00,4,37 days 12:30:00,5,97.469773,A100,y
1,run-pipeline-gpu-8g5pb-2879794351,task,PROJECT_13,ns-16904409679649498,NaN,2023-11-08 12:15:00+00:00,2023-11-22 02:15:00+00:00,8,13 days 14:00:00,5,93.878238,A100,y
6,run-pipeline-gpu-qr2fk-3879037461,task,PROJECT_5,ns-1690440796148727,NaN,2023-11-08 12:15:00+00:00,2023-11-13 00:15:00+00:00,1,4 days 12:00:00,4,67.799001,A100,y
7,run-pipeline-gpu-cqxgz-1528876722,task,PROJECT_13,ns-16904409679649498,TEAM_1,2023-11-08 12:15:00+00:00,2023-11-27 07:30:00+00:00,16,18 days 19:15:00,3,53.086390,A100,y
17,run-pipeline-gpu-8z2pg-3801962406,task,PROJECT_1,ns-16904458533318198,TEAM_2,2023-11-08 12:15:00+00:00,2023-11-09 09:15:00+00:00,2,0 days 21:00:00,5,95.259751,A30,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,instance-2141-0,instance,PROJECT_14,ns-1693886715473508,TEAM_6,2023-12-01 09:00:00+00:00,2023-12-02 09:00:00+00:00,1,1 days 00:00:00,1,0.000000,A30,n
2039,instance-2220-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-08 08:30:00+00:00,2023-12-09 08:30:00+00:00,1,1 days 00:00:00,1,0.000000,A30,n
2044,instance-2260-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-18 10:00:00+00:00,2023-12-18 10:15:00+00:00,1,0 days 00:15:00,1,0.000000,A30,y
2045,instance-2142-0,instance,PROJECT_14,ns-1693886715473508,TEAM_5,2023-12-20 00:30:00+00:00,2023-12-21 00:30:00+00:00,1,1 days 00:00:00,1,0.352398,A30,y


In [14]:
task_total.to_csv('job_flow_total(task,flavor,single).csv', index=False)